In [2]:
import sys
sys.path.append("../src")
import os

import pandas as pd
import numpy as np
from etf_transformations import *
from pathlib import Path

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, TimeSeriesSplit
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostClassifier
from scipy.stats import uniform, randint
from sklearn.preprocessing import FunctionTransformer

In [3]:
df = pd.read_csv("../data/preprocessed/separate_stocks_finbert_with_emb.csv")


In [4]:
df.head()

,title,date,stock,exchange,timestamp_et,time_et,date_et,news_window,sector,is_trading_day,...,emb_374,emb_375,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383
0,Going Against the Herd,2009-04-29 08:48:00-04:00,A,NYSE,2009-04-29 08:48:00-04:00,08:48:00,2009-04-29,overnight,Health Care,True,...,0.044927,-0.013433,-0.041791,-0.007552,-0.043562,0.064962,0.024598,-0.036362,0.005609,0.006723
1,super-trades - Two 52 week highs and others ab...,2009-06-01 22:15:00-04:00,A,NYSE,2009-06-01 22:15:00-04:00,22:15:00,2009-06-01,overnight,Health Care,True,...,-0.005445,0.072854,-0.033382,0.050525,-0.021976,-0.061133,0.071026,-0.104717,-0.154968,0.054481
2,Skystar Bio-Pharmaceutical Announces Expansion...,2009-07-14 08:14:00-04:00,A,NYSE,2009-07-14 08:14:00-04:00,08:14:00,2009-07-14,overnight,Health Care,True,...,0.022711,0.004054,-0.060530,-0.050457,0.027259,-0.058011,0.011103,-0.155406,0.021580,0.044305
3,The Gold/Silver Ratio From 1300 to 1900...And Now,2009-07-30 18:01:00-04:00,A,NYSE,2009-07-30 18:01:00-04:00,18:01:00,2009-07-30,after_close,Health Care,True,...,0.024095,-0.093397,-0.001406,-0.012596,-0.084371,0.017338,0.076390,-0.135631,-0.070295,0.050183
4,"A Simulation of China's 2,$$$,$$$,$$$,$$$ Dive...",2009-08-04 06:45:00-04:00,A,NYSE,2009-08-04 06:45:00-04:00,06:45:00,2009-08-04,overnight,Health Care,True,...,0.101733,0.006196,-0.056580,0.048957,-0.050295,0.028605,0.017734,-0.075114,-0.104239,0.057186


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324380 entries, 0 to 324379
Columns: 414 entries, title to emb_383
dtypes: bool(1), float64(397), int64(5), object(11)
memory usage: 1022.4+ MB


In [6]:
df.describe()

,articles_same_stock_day,articles_same_sector_day,price,price_prev_day,price_next_day,Return,Sign,Return_next_day,AbsReturn_next_day,Sign_next_day,...,emb_374,emb_375,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383
count,324380.000000,324380.000000,324380.000000,324328.000000,324380.000000,324328.000000,324328.000000,324380.000000,324380.000000,324380.000000,...,324380.000000,324380.000000,324380.000000,324380.000000,324380.000000,324380.000000,324380.000000,324380.000000,324380.000000,324380.000000
mean,4.708453,22.725026,61.474066,61.432637,61.492615,0.001804,0.065246,0.000275,0.016125,0.514132,...,0.011564,0.002556,-0.040031,0.002193,-0.034535,-0.010964,0.017697,-0.101920,-0.012769,0.031535
std,8.322607,17.502838,86.649847,86.623722,86.704153,0.041623,0.995400,0.027286,0.022013,0.499801,...,0.044356,0.048861,0.043218,0.043489,0.046107,0.047714,0.049687,0.052307,0.052680,0.043921
min,1.000000,1.000000,0.268174,0.260839,0.265060,-0.529074,-1.000000,-0.753054,0.000000,0.000000,...,-0.195600,-0.207168,-0.206184,-0.214126,-0.225958,-0.208383,-0.252924,-0.282594,-0.226734,-0.185033
25%,1.000000,11.000000,23.306425,23.312603,23.311546,-0.011854,-1.000000,-0.009459,0.004397,0.000000,...,-0.018748,-0.031447,-0.069229,-0.027910,-0.065588,-0.045722,-0.014686,-0.139414,-0.049154,0.001701
50%,2.000000,18.000000,40.444548,40.429642,40.448544,0.001393,1.000000,0.000557,0.010054,1.000000,...,0.011940,0.001911,-0.041255,0.003210,-0.036245,-0.013295,0.020302,-0.104597,-0.014635,0.033393
75%,5.000000,30.000000,71.152288,71.100891,71.190870,0.015035,1.000000,0.010590,0.019831,1.000000,...,0.042195,0.037680,-0.013185,0.032917,-0.004270,0.022212,0.051504,-0.067490,0.021864,0.062747
max,169.000000,186.000000,3983.600098,4030.000000,4021.750000,0.745933,1.000000,0.557289,0.753054,1.000000,...,0.198144,0.232822,0.162369,0.205478,0.197540,0.232106,0.242711,0.157245,0.219789,0.189359


In [7]:
df.isna().sum()

title           0
date            0
stock           0
exchange        0
timestamp_et    0
               ..
emb_379         0
emb_380         0
emb_381         0
emb_382         0
emb_383         0
Length: 414, dtype: int64

In [8]:
df.nunique()

title           224675
date            216210
stock              381
exchange             2
timestamp_et    216210
                 ...  
emb_379         227592
emb_380         227532
emb_381         226385
emb_382         227603
emb_383         227347
Length: 414, dtype: int64

In [9]:
df.shape

(324380, 414)

In [10]:
df["Sign_next_day"] = (df["Sign_next_day"] == 1).astype(int)
df.head()

,title,date,stock,exchange,timestamp_et,time_et,date_et,news_window,sector,is_trading_day,...,emb_374,emb_375,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383
0,Going Against the Herd,2009-04-29 08:48:00-04:00,A,NYSE,2009-04-29 08:48:00-04:00,08:48:00,2009-04-29,overnight,Health Care,True,...,0.044927,-0.013433,-0.041791,-0.007552,-0.043562,0.064962,0.024598,-0.036362,0.005609,0.006723
1,super-trades - Two 52 week highs and others ab...,2009-06-01 22:15:00-04:00,A,NYSE,2009-06-01 22:15:00-04:00,22:15:00,2009-06-01,overnight,Health Care,True,...,-0.005445,0.072854,-0.033382,0.050525,-0.021976,-0.061133,0.071026,-0.104717,-0.154968,0.054481
2,Skystar Bio-Pharmaceutical Announces Expansion...,2009-07-14 08:14:00-04:00,A,NYSE,2009-07-14 08:14:00-04:00,08:14:00,2009-07-14,overnight,Health Care,True,...,0.022711,0.004054,-0.060530,-0.050457,0.027259,-0.058011,0.011103,-0.155406,0.021580,0.044305
3,The Gold/Silver Ratio From 1300 to 1900...And Now,2009-07-30 18:01:00-04:00,A,NYSE,2009-07-30 18:01:00-04:00,18:01:00,2009-07-30,after_close,Health Care,True,...,0.024095,-0.093397,-0.001406,-0.012596,-0.084371,0.017338,0.076390,-0.135631,-0.070295,0.050183
4,"A Simulation of China's 2,$$$,$$$,$$$,$$$ Dive...",2009-08-04 06:45:00-04:00,A,NYSE,2009-08-04 06:45:00-04:00,06:45:00,2009-08-04,overnight,Health Care,True,...,0.101733,0.006196,-0.056580,0.048957,-0.050295,0.028605,0.017734,-0.075114,-0.104239,0.057186


In [12]:
df["Sign_next_day"].value_counts(normalize=True)


Sign_next_day
1    0.514132
0    0.485868
Name: proportion, dtype: float64

In [11]:
df_catboost = df.copy()
df_catboost.head()

,title,date,stock,exchange,timestamp_et,time_et,date_et,news_window,sector,is_trading_day,...,emb_374,emb_375,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383
0,Going Against the Herd,2009-04-29 08:48:00-04:00,A,NYSE,2009-04-29 08:48:00-04:00,08:48:00,2009-04-29,overnight,Health Care,True,...,0.044927,-0.013433,-0.041791,-0.007552,-0.043562,0.064962,0.024598,-0.036362,0.005609,0.006723
1,super-trades - Two 52 week highs and others ab...,2009-06-01 22:15:00-04:00,A,NYSE,2009-06-01 22:15:00-04:00,22:15:00,2009-06-01,overnight,Health Care,True,...,-0.005445,0.072854,-0.033382,0.050525,-0.021976,-0.061133,0.071026,-0.104717,-0.154968,0.054481
2,Skystar Bio-Pharmaceutical Announces Expansion...,2009-07-14 08:14:00-04:00,A,NYSE,2009-07-14 08:14:00-04:00,08:14:00,2009-07-14,overnight,Health Care,True,...,0.022711,0.004054,-0.060530,-0.050457,0.027259,-0.058011,0.011103,-0.155406,0.021580,0.044305
3,The Gold/Silver Ratio From 1300 to 1900...And Now,2009-07-30 18:01:00-04:00,A,NYSE,2009-07-30 18:01:00-04:00,18:01:00,2009-07-30,after_close,Health Care,True,...,0.024095,-0.093397,-0.001406,-0.012596,-0.084371,0.017338,0.076390,-0.135631,-0.070295,0.050183
4,"A Simulation of China's 2,$$$,$$$,$$$,$$$ Dive...",2009-08-04 06:45:00-04:00,A,NYSE,2009-08-04 06:45:00-04:00,06:45:00,2009-08-04,overnight,Health Care,True,...,0.101733,0.006196,-0.056580,0.048957,-0.050295,0.028605,0.017734,-0.075114,-0.104239,0.057186


In [13]:
df_catboost["trading_date"] = pd.to_datetime(df_catboost["trading_date"]).dt.normalize()
df_catboost = df_catboost.sort_values("trading_date").reset_index(drop=True)

In [14]:
df_catboost["date_et"] = pd.to_datetime(df_catboost["date_et"], errors="coerce")
df_catboost["day_of_week"] = df_catboost["date_et"].dt.dayofweek
df_catboost["month"] = df_catboost["date_et"].dt.month
df_catboost.head()

,title,date,stock,exchange,timestamp_et,time_et,date_et,news_window,sector,is_trading_day,...,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383,day_of_week,month
0,Going Against the Herd,2009-04-29 08:48:00-04:00,A,NYSE,2009-04-29 08:48:00-04:00,08:48:00,2009-04-29,overnight,Health Care,True,...,-0.041791,-0.007552,-0.043562,0.064962,0.024598,-0.036362,0.005609,0.006723,2,4
1,MRM a $15-$20+ stock - FIT new information - J...,2009-05-27 21:35:00-04:00,RF,NYSE,2009-05-27 21:35:00-04:00,21:35:00,2009-05-27,overnight,Financials,True,...,-0.018864,0.009394,0.022785,-0.050416,0.005210,-0.186018,-0.129083,0.030637,2,5
2,super-trades - Two 52 week highs and others ab...,2009-06-01 22:15:00-04:00,A,NYSE,2009-06-01 22:15:00-04:00,22:15:00,2009-06-01,overnight,Health Care,True,...,-0.033382,0.050525,-0.021976,-0.061133,0.071026,-0.104717,-0.154968,0.054481,0,6
3,super-trades - Two 52 week highs and others ab...,2009-06-01 22:15:00-04:00,DD,NYSE,2009-06-01 22:15:00-04:00,22:15:00,2009-06-01,overnight,Materials,True,...,-0.033382,0.050525,-0.021976,-0.061133,0.071026,-0.104717,-0.154968,0.054481,0,6
4,"Last 3 picks all took off this week 67%,62%,30...",2009-06-05 23:28:00-04:00,DD,NYSE,2009-06-05 23:28:00-04:00,23:28:00,2009-06-05,overnight,Materials,True,...,-0.048096,-0.003201,-0.076592,-0.035744,-0.010230,-0.087495,-0.078073,0.090731,4,6


In [19]:
original_features = [
    "stock", "exchange", "time_et",
    "news_window", "sector", "is_trading_day",
    "articles_same_stock_day", "articles_same_sector_day",
    "price", "vol_5",
    "positive", "neutral", "negative",
    "finbert_label", "finbert_confidence",
    "day_of_week", "month"
]

embedding_features = [f"emb_{i}" for i in range(384)]
target = "Sign_next_day"

df = df_catboost.copy()

X = df[original_features + embedding_features]
y = df[target].astype(int)

print(f"Target distribution: {y.value_counts().to_dict()}")

Target distribution: {1: 166774, 0: 157606}


In [20]:
cat_cols = ["stock","exchange","time_et","news_window","sector","finbert_label","is_trading_day"]
cat_idx = [X.columns.get_loc(c) for c in cat_cols]


In [23]:
emb_cols = [c for c in X.columns if c.startswith("emb_")]

pca = PCA(n_components=200, random_state=12)
E_pca = pca.fit_transform(X[emb_cols].values)

print(f"Explained variance: {pca.explained_variance_ratio_.sum():.4f}")

Explained variance: 0.9287


In [24]:
pca_cols = [f"pca_{i}" for i in range(E_pca.shape[1])]
X_pca = pd.DataFrame(E_pca, columns=pca_cols, index=X.index)

X_base = X.drop(columns=emb_cols)

X_final = pd.concat([X_base, X_pca], axis=1)

print("X_final shape:", X_final.shape)

X_final shape: (324380, 217)


In [25]:
cat_cols = [
    "stock", "exchange", "time_et",
    "news_window", "sector",
    "finbert_label", "is_trading_day"
]

cat_idx = [X_final.columns.get_loc(c) for c in cat_cols]
print("Categorical indices:", cat_idx)

Categorical indices: [0, 1, 2, 3, 4, 13, 5]


In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

base_model = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=12,
    verbose=0,
    allow_writing_files=False
)

param_distributions = {
    "depth": randint(4, 9),
    "learning_rate": uniform(0.01, 0.12),
    "iterations": randint(600, 1600),
    "l2_leaf_reg": uniform(1.0, 20.0),
    "random_strength": uniform(0.0, 5.0),
    "min_data_in_leaf": randint(20, 120),
    "rsm": uniform(0.6, 0.4),
    "bootstrap_type": ["Bayesian", "MVS"],
    "bagging_temperature": uniform(0.0, 1.0),
}

search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_distributions,
    n_iter=30,
    scoring="roc_auc",
    cv=tscv,
    n_jobs=-1,
    verbose=2,
    random_state=12,
    return_train_score=True
)

search.fit(X_final, y, cat_features=cat_idx)

print("Best CV AUC:", search.best_score_)
print("Best params:", search.best_params_)

In [ ]:
cut = int(len(X_final) * 0.8)
X_train, X_test = X_final.iloc[:cut], X_final.iloc[cut:]
y_train, y_test = y.iloc[:cut], y.iloc[cut:]

best_model = search.best_estimator_
best_model.set_params(verbose=200)

best_model.fit(X_train, y_train, cat_features=cat_idx)

proba = best_model.predict_proba(X_test)[:, 1]
pred = (proba >= 0.5).astype(int)

print("Holdout Test AUC:", roc_auc_score(y_test, proba))
print("\nConfusion matrix:\n", confusion_matrix(y_test, pred))
print("\nReport:\n", classification_report(y_test, pred, digits=4))


In [ ]:
df_catboost["trading_date"].is_monotonic_increasing
